In [73]:
import numpy as np 
import pandas as pd

from sklearn.metrics import hamming_loss,accuracy_score,classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier

import os

from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings("ignore")

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv',header=None)
train_target_df = pd.read_csv('trainLabels.csv')
submission_df = pd.read_csv('sampleSubmission_small.csv')

In [3]:
print("train_df --> ",train_df.shape)
print("test_df --> ",test_df.shape)
print("train_target_df --> ",train_target_df.shape)
print("submission_df --> ",submission_df.shape)

train_df -->  (9999, 146)
test_df -->  (2000, 146)
train_target_df -->  (49999, 34)
submission_df -->  (990, 2)


In [4]:
test_df.head()

,0,1,2,3,4,5,6,7,8,9,...,136,137,138,139,140,141,142,143,144,145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [5]:
train_target_df.head()

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
test_df.columns=['id',
                 'x1','x2','x3','x4','x5','x6','x7','x8','x9','x10',
                 'x11','x12','x13','x14','x15','x16','x17','x18','x19','x20',
                 'x21','x22','x23','x24','x25','x26','x27','x28','x29','x30',
                 'x31','x32','x33','x34','x35','x36','x37','x38','x39','x40',
                 'x41','x42','x43','x44','x45','x46','x47','x48','x49','x50',
                 'x51','x52','x53','x54','x55','x56','x57','x58','x59','x60',
                 'x61','x62','x63','x64','x65','x66','x67','x68','x69','x70',
                 'x71','x72','x73','x74','x75','x76','x77','x78','x79','x80',
                 'x81','x82','x83','x84','x85','x86','x87','x88','x89','x90',
                 'x91','x92','x93','x94','x95','x96','x97','x98','x99','x100',
                 'x101','x102','x103','x104','x105','x106','x107','x108','x109','x110',
                 'x111','x112','x113','x114','x115','x116','x117','x118','x119','x120',
                 'x121','x122','x123','x124','x125','x126','x127','x128','x129','x130',
                 'x131','x132','x133','x134','x135','x136','x137','x138','x139','x140',
                 'x141','x142','x143','x144','x145']

In [7]:
test_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1698001,NO,NO,5KaYd5siHnBD/IjH8BF1fPz5zrCADHZia/Lrhlyxkvc=,FzMc/XY2ETaomhy8gPc9UL8LRkEnQA56+/wVF1fogk8=,1.414798,0.000000,1.000000,0.000000,0.202060,...,0.0,1.000000,1262,892,NO,NO,NO,0,0.089686,0.193344
1,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,...,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
2,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.000000,1.000000,0.000000,0.703088,...,-1.0,1.000000,1263,892,NO,NO,NO,8,0.557175,0.693587
3,1698004,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.870000,4672,3306,YES,NO,YES,0,0.870538,0.405822
4,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.129212,0.087020,0.814240,1.112804,0.874318,...,0.0,0.870000,4400,3413,YES,NO,YES,2,0.224729,0.870909


In [8]:
train_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306,4676,YES,NO,YES,2,0.375535,0.464610
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678,3306,YES,NO,YES,4,0.741682,0.593630
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678,3306,NO,NO,NO,1,0.776467,0.493159
3,4,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306,4678,NO,NO,YES,3,0.168234,0.546582
4,5,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263,892,NO,NO,NO,1,0.246637,0.361045


In [9]:
#All rows that has atleast one NAN value
train_df[train_df.isna().any(axis=1)]

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.51,4678,3306,YES,NO,YES,4,0.741682,0.593630
8,9,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.82,4675,3304,YES,NO,YES,1,0.923426,0.702032
12,13,NO,NO,vIR+5O6D5pKgFw/CFbDtHBZ/AhaNMnxLM4BTIwXUres=,gNVKedeHcKHyWnIMnBCE9t9XOschRQCU8/XfcaY0mMk=,0.440411,0.083787,0.776165,0.253781,0.210774,...,0.0,0.74,4678,3306,YES,NO,YES,0,0.029643,0.199230
25,26,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1188,918,YES,NO,YES,5,0.898693,0.502525
26,27,NO,NO,EKynK11V/TnbPzoD8gy12kkm85wTNzN4kciauQQOHO8=,LXzIvxHIblNu5ThzWnaNlPxZrF3BRg4q4JWZPXQ5Z40=,1.415919,0.000000,1.000000,0.000000,0.889153,...,0.0,1.00,1263,892,YES,NO,YES,0,0.032511,0.883610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9978,9979,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1262,892,YES,NO,YES,1,0.838565,0.194136
9983,9984,NO,NO,/A8gcPhvt4VEgUwvb8EOaJsx1W/EJcRZvR+LpZQCG5I=,hCXwO/JldK5zcd9ejOD1FwmEgCf96eTdEVy7OtY2Y2g=,1.414798,0.000000,1.000000,0.000000,0.798732,...,0.0,1.00,1262,892,YES,NO,YES,1,0.454036,0.787639
9985,9986,NO,NO,jSCyZ7TLJ2iF5csw4WvfdUDPg0rilDps2Bz6vPTD6K4=,k/m0ZdAeAo2VCPgrn4lo9Aow1dirDyCAmhbX5fvZd7M=,0.793166,0.074085,0.767765,0.617780,0.479581,...,0.0,0.91,4677,3307,NO,NO,NO,0,0.167523,0.470601
9991,9992,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.00,1263,893,YES,YES,YES,4,0.731243,0.284244


<b>Preprocessing the Data</b>

In [10]:
missing_percent_training_data=train_df.isna().sum().sort_values(ascending=False)*100/len(train_df)
missing_percent_training_data

x26     14.261426
x12     14.261426
x2      14.261426
x3      14.261426
x4      14.261426
          ...    
x53      0.000000
x52      0.000000
x51      0.000000
x50      0.000000
x145     0.000000
Length: 146, dtype: float64

In [11]:
missing_percent_train_target_data=train_target_df.isna().sum().sort_values(ascending=False)*100/len(train_target_df)
missing_percent_train_target_data

id     0.0
y25    0.0
y19    0.0
y20    0.0
y21    0.0
y22    0.0
y23    0.0
y24    0.0
y26    0.0
y1     0.0
y27    0.0
y28    0.0
y29    0.0
y30    0.0
y31    0.0
y32    0.0
y18    0.0
y17    0.0
y16    0.0
y15    0.0
y14    0.0
y13    0.0
y12    0.0
y11    0.0
y10    0.0
y9     0.0
y8     0.0
y7     0.0
y6     0.0
y5     0.0
y4     0.0
y3     0.0
y2     0.0
y33    0.0
dtype: float64

In [12]:
missing_percent_test_data=test_df.isna().sum().sort_values(ascending=False)*100/len(test_df)
missing_percent_test_data

x26     14.7
x12     14.7
x2      14.7
x3      14.7
x4      14.7
        ... 
x53      0.0
x52      0.0
x51      0.0
x50      0.0
x145     0.0
Length: 146, dtype: float64

In [13]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numeric_columns=train_df.select_dtypes(include=numerics).columns.to_list()
len(numeric_columns) 

86

In [14]:
object_columns=train_df.select_dtypes(include=object).columns
len(object_columns)

60

In [15]:
train_df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [16]:
train_df[object_columns]=train_df[object_columns].astype(str)
test_df[object_columns]=test_df[object_columns].astype(str)

In [17]:
train_df[object_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 60 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   x1      9999 non-null   object
 1   x2      9999 non-null   object
 2   x3      9999 non-null   object
 3   x4      9999 non-null   object
 4   x10     9999 non-null   object
 5   x11     9999 non-null   object
 6   x12     9999 non-null   object
 7   x13     9999 non-null   object
 8   x14     9999 non-null   object
 9   x24     9999 non-null   object
 10  x25     9999 non-null   object
 11  x26     9999 non-null   object
 12  x30     9999 non-null   object
 13  x31     9999 non-null   object
 14  x32     9999 non-null   object
 15  x33     9999 non-null   object
 16  x34     9999 non-null   object
 17  x35     9999 non-null   object
 18  x41     9999 non-null   object
 19  x42     9999 non-null   object
 20  x43     9999 non-null   object
 21  x44     9999 non-null   object
 22  x45     9999 non-null   

In [18]:
boolean_cols = []
alphanumeric_cols = []

for col in object_columns:
    if train_df[col][0].isupper()==True:
        boolean_cols.append(col)
    else:
        alphanumeric_cols.append(col)

In [19]:
alphanumeric_cols

['x3', 'x4', 'x34', 'x35', 'x61', 'x64', 'x65', 'x91', 'x94', 'x95']

In [20]:
train_df['x3'].value_counts()

nan                                             1426
MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=     290
X/hdUOVR5KuExVGLzjhLcM2CyIqym9t0Nh+ZX05M+1w=     100
3acC0FNTp8BjTkXkLzy9LJSB0FmLpzsJhDtE3riGZ9A=      70
zq3MyXbqpnH+txFzhEWomNfZyeLXubUbZwdaRvKXLs4=      68
                                                ... 
xjmTLdl3cvznW2FwUHzAy1isqqQ/MM75nhLGKaSAEFY=       1
32TnZIoTdTuMO/QEiTndO7R2De+TXxjzsCcGB68PxSs=       1
Zpdzn+mU6g3vVZMqG3IiKWgYk08wwV0eyt7zxX+yew8=       1
l+LC+epkima2ujQczUHf+UXtXJfTXY7r7PDdGSSdSvg=       1
9Avs0tL1zvH7Xx41z2UqrXs11/4IWLmqRAodLt/SKjQ=       1
Name: x3, Length: 4752, dtype: int64

In [21]:
train_df[boolean_cols]=train_df[boolean_cols].astype('category')
test_df[boolean_cols]=test_df[boolean_cols].astype('category')

In [22]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 146 entries, id to x145
dtypes: category(50), float64(55), int64(31), object(10)
memory usage: 7.8+ MB


In [23]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Columns: 146 entries, id to x145
dtypes: category(50), float64(55), int64(31), object(10)
memory usage: 1.6+ MB


In [24]:
num_imputer=SimpleImputer(strategy='mean')

In [25]:
obj_imputer=SimpleImputer(strategy='most_frequent')

In [26]:
num_imputer.fit(train_df[numeric_columns])
train_df[numeric_columns]=num_imputer.transform(train_df[numeric_columns])

In [27]:
num_imputer.fit(test_df[numeric_columns])

SimpleImputer()

In [28]:
test_df[numeric_columns]=num_imputer.transform(test_df[numeric_columns])

In [29]:
train_df[numeric_columns].isna().sum().sort_values(ascending=False)

id      0
x98     0
x111    0
x110    0
x109    0
       ..
x50     0
x49     0
x48     0
x47     0
x145    0
Length: 86, dtype: int64

In [30]:
test_df[numeric_columns].isna().sum().sort_values(ascending=False)

id      0
x98     0
x111    0
x110    0
x109    0
       ..
x50     0
x49     0
x48     0
x47     0
x145    0
Length: 86, dtype: int64

In [31]:
train_df.replace('nan', None, inplace=True)
test_df.replace('nan', None, inplace=True)

In [32]:
train_df.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1.0,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,...,0.0,0.810,3306.0,4676.0,YES,NO,YES,2.0,0.375535,0.464610
1,2.0,NaN,NaN,None,None,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.510,4678.0,3306.0,YES,NO,YES,4.0,0.741682,0.593630
2,3.0,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,...,0.0,0.850,4678.0,3306.0,NO,NO,NO,1.0,0.776467,0.493159
3,4.0,YES,NO,BfrqME7vdLw3suQp6YAT16W2piNUmpKhMzuDrVrFQ4w=,YGCdISifn4fLao/ASKdZFhGIq23oqzfSbUVb6px1pig=,0.653912,0.041471,0.940787,0.090851,0.556564,...,0.0,0.945,3306.0,4678.0,NO,NO,YES,3.0,0.168234,0.546582
4,5.0,NO,NO,RTjsrrR8DTlJyaIP9Q3Z8s0zseqlVQTrlSe97GCWfbk=,3yK2OPj1uYDsoMgsxsjY1FxXkOllD8Xfh20VYGqT+nU=,1.415919,0.000000,1.000000,0.000000,0.375297,...,0.0,1.000,1263.0,892.0,NO,NO,NO,1.0,0.246637,0.361045


In [33]:
for col in boolean_cols:
    train_df[col].replace({'YES':1, 'NO':0},inplace=True)
    test_df[col].replace({'YES':1, 'NO':0},inplace=True)

In [34]:
train_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
9995,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
9996,1,1,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,1
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [35]:
obj_imputer.fit(train_df[boolean_cols])
train_df[boolean_cols]=obj_imputer.transform(train_df[boolean_cols])

In [36]:
obj_imputer.fit(test_df[boolean_cols])
test_df[boolean_cols]=obj_imputer.transform(test_df[boolean_cols])

In [37]:
test_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,0
2,0,0,0,0,1,1,1,0,0,0,...,0,0,0,0,1,1,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,1,1,0,1
1996,0,0,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,1
1997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1998,0,0,0,0,1,0,0,0,0,0,...,0,1,1,0,1,0,1,0,0,0


In [38]:
train_df[boolean_cols]

,x1,x2,x10,x11,x12,x13,x14,x24,x25,x26,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,0,0,1
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
9995,0,0,1,0,0,0,0,1,0,1,...,0,1,1,0,0,0,0,1,0,1
9996,1,1,1,0,0,0,0,1,0,1,...,0,0,1,0,0,0,0,1,0,1
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [39]:
train_df[boolean_cols].isna().sum().sort_values(ascending=False)

x1      0
x104    0
x74     0
x75     0
x85     0
x86     0
x87     0
x92     0
x93     0
x101    0
x102    0
x103    0
x105    0
x2      0
x115    0
x116    0
x117    0
x126    0
x127    0
x128    0
x129    0
x130    0
x140    0
x141    0
x73     0
x72     0
x71     0
x63     0
x10     0
x11     0
x12     0
x13     0
x14     0
x24     0
x25     0
x26     0
x30     0
x31     0
x32     0
x33     0
x41     0
x42     0
x43     0
x44     0
x45     0
x55     0
x56     0
x57     0
x62     0
x142    0
dtype: int64

In [40]:
train_df[boolean_cols]=train_df[boolean_cols].astype('int')
test_df[boolean_cols]=test_df[boolean_cols].astype('int')

In [41]:
train_df[numeric_columns].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 86 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      9999 non-null   float64
 1   x5      9999 non-null   float64
 2   x6      9999 non-null   float64
 3   x7      9999 non-null   float64
 4   x8      9999 non-null   float64
 5   x9      9999 non-null   float64
 6   x15     9999 non-null   float64
 7   x16     9999 non-null   float64
 8   x17     9999 non-null   float64
 9   x18     9999 non-null   float64
 10  x19     9999 non-null   float64
 11  x20     9999 non-null   float64
 12  x21     9999 non-null   float64
 13  x22     9999 non-null   float64
 14  x23     9999 non-null   float64
 15  x27     9999 non-null   float64
 16  x28     9999 non-null   float64
 17  x29     9999 non-null   float64
 18  x36     9999 non-null   float64
 19  x37     9999 non-null   float64
 20  x38     9999 non-null   float64
 21  x39     9999 non-null   float64
 22  

In [42]:
exclude_column = 'id'
numeric_columns = list(filter(lambda col: col != exclude_column, numeric_columns))

In [43]:
X_train = train_df[numeric_columns+boolean_cols]
X_test  = test_df[numeric_columns+boolean_cols]
Y_train = train_target_df.loc[:9998,'y1':]

In [44]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)

(9999, 135)
(9999, 33)
(2000, 135)


<b>Training the Model & Predicting the Classes</b>

In [45]:
X_train

,x5,x6,x7,x8,x9,x15,x16,x17,x18,x19,...,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,0.576561,0.073139,0.481394,0.115697,0.472474,42.0,0.396065,3.0,6.0,0.991018,...,0,1,1,0,0,0,0,1,0,1
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0,0,1,0,0,0,0,1,0,1
2,1.341803,0.051422,0.935572,0.041440,0.501710,2.0,0.838475,3.0,5.0,0.966122,...,0,0,0,0,1,0,0,0,0,0
3,0.653912,0.041471,0.940787,0.090851,0.556564,37.0,0.127405,8.0,15.0,0.959171,...,0,1,1,0,0,0,0,0,0,1
4,1.415919,0.000000,1.000000,0.000000,0.375297,1.0,0.523543,4.0,11.0,0.963004,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1.207136,0.082855,0.918960,0.313880,0.495189,10.0,0.512852,3.0,7.0,0.967342,...,0,0,1,0,0,0,0,1,0,1
9995,1.414798,0.000000,1.000000,0.000000,0.357369,6.0,0.864350,4.0,17.0,0.974215,...,0,1,1,0,0,0,0,1,0,1
9996,1.413677,0.000000,1.000000,0.000000,0.668517,3.0,0.865471,6.0,7.0,0.939462,...,0,0,1,0,0,0,0,1,0,1
9997,1.294118,0.000000,1.000000,0.000000,0.570707,7.0,0.400871,8.0,11.0,0.949891,...,0,0,0,0,1,0,0,0,0,0


In [104]:
def hash_columns(df, columns_to_hash):
    hashed_df = pd.DataFrame()

    for column in alphanumeric_cols:
        hashed_df['hashed_'+column] = df[column].apply(hash)

    return hashed_df

hashed_train_df = hash_columns(train_df,alphanumeric_cols)

hashed_test_df = hash_columns(test_df,alphanumeric_cols)

In [106]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

scaled_hashed_train_df = pd.DataFrame(scaler.fit_transform(hashed_train_df), columns=hashed_train_df.columns)
scaled_hashed_test_df = pd.DataFrame(scaler.transform(hashed_test_df), columns=hashed_test_df.columns)

In [113]:
scaled_hashed_train_df

,hashed_x3,hashed_x4,hashed_x34,hashed_x35,hashed_x61,hashed_x64,hashed_x65,hashed_x91,hashed_x94,hashed_x95
0,0.998582,0.649164,0.133303,0.252316,0.073837,0.202415,0.358593,0.140474,0.189442,0.587728
1,0.000000,0.000000,0.749177,0.005548,0.911451,0.163048,0.012171,0.407508,0.190970,0.378864
2,0.325034,0.378230,0.966884,0.374118,0.632540,0.010679,0.452857,0.378285,0.463109,0.108510
3,0.443603,0.879564,0.443751,0.879817,0.707292,0.026687,0.879817,0.561988,0.026687,0.881038
4,0.152676,0.778024,0.726779,0.428495,0.401796,0.200622,0.199940,0.920378,0.351683,0.796467
...,...,...,...,...,...,...,...,...,...,...
9994,0.170524,0.747091,0.149055,0.737265,0.172968,0.202413,0.778248,0.963997,0.186588,0.367049
9995,0.657034,0.407437,0.467830,0.452857,0.121231,0.467830,0.452857,0.302091,0.467830,0.453486
9996,0.489302,0.452727,0.901486,0.451721,0.489304,0.103829,0.407554,0.452823,0.168327,0.916722
9997,0.630905,0.499872,0.100121,0.747305,0.632355,0.100121,0.747305,0.515283,0.800825,0.200218


In [116]:
X_train_hash = pd.concat([X_train,scaled_hashed_train_df], axis=1)
X_test_hash = pd.concat([X_test,scaled_hashed_test_df], axis=1)

In [118]:
print(X_train_hash.shape)
print(Y_train.shape)
print(X_test_hash.shape)

(9999, 145)
(9999, 33)
(2000, 145)


In [120]:
model = OneVsRestClassifier(RandomForestClassifier())
model.fit(X_train_hash,Y_train)
Y_pred_hash = model.predict(X_train_hash)
ham_loss = hamming_loss(Y_train, Y_pred_hash)
print("Hamming Loss for Training Data",ham_loss)

C:\Users\TEJKIRAN\anaconda3\lib\site-packages\sklearn\multiclass.py:79: UserWarning: Label not 13 is present in all training examples.
  warnings.warn(


Hamming Loss for Training Data 6.061212181824243e-06


In [121]:
print("Accuracy Score for Training Data",accuracy_score(Y_train,Y_pred_hash))

Accuracy Score for Training Data 0.9997999799979999


In [122]:
classification_rep = classification_report(Y_train,Y_pred)

print('Classification Report:\n', classification_rep)

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        60
           1       1.00      1.00      1.00         6
           2       1.00      1.00      1.00       220
           3       1.00      1.00      1.00       156
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00       761
           6       1.00      1.00      1.00       350
           7       1.00      1.00      1.00         6
           8       1.00      1.00      1.00       774
           9       0.99      1.00      1.00       156
          10       1.00      1.00      1.00         6
          11       1.00      1.00      1.00       742
          12       1.00      1.00      1.00       132
          13       0.00      0.00      0.00         0
          14       1.00      1.00      1.00        27
          15       1.00      1.00      1.00       112
          16       1.00      1.00      1.00         1
   

C:\Users\TEJKIRAN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\TEJKIRAN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
test_df.id

0       1698001.0
1       1698002.0
2       1698003.0
3       1698004.0
4       1698005.0
          ...    
1995    1699996.0
1996    1699997.0
1997    1699998.0
1998    1699999.0
1999    1700000.0
Name: id, Length: 2000, dtype: float64

In [57]:
target=[]
for num in test_df.id:
    for i in range(1,34):
        target.append(str(num)+'_y'+str(i))

In [58]:
len(target)

66000

In [59]:
submission_df=pd.DataFrame(target,columns=['id'])

In [60]:
submission_df

,id
0,1698001.0_y1
1,1698001.0_y2
2,1698001.0_y3
3,1698001.0_y4
4,1698001.0_y5
...,...
65995,1700000.0_y29
65996,1700000.0_y30
65997,1700000.0_y31
65998,1700000.0_y32


In [123]:
# prediction for Test data
predictions_hash = model.predict(X_test_hash)

In [124]:
prediction_hash_list=[]
for i in range(len(predictions_hash)):
    prediction_hash_list.extend(predictions_hash.tolist()[i])

In [125]:
pred_hash_df=pd.DataFrame(prediction_hash_list,columns=['pred'])

In [126]:
test_prediction_hash_df=pd.concat([submission_df,pred_hash_df],axis=1)

In [127]:
test_prediction_hash_df.to_csv('test_prediction_final.csv',index=None)